In [1]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [2]:
"""!pip install datasets
!pip install transformers
!pip install sentence_transformers
!pip install peft
!pip install huggingface_hub
!pip install --upgrade accelerate
!pip install --upgrade bitsandbytes
!pip install --upgrade torchmetrics
!pip install --upgrade deepspeed
"""

'!pip install datasets\n!pip install transformers\n!pip install sentence_transformers\n!pip install peft\n!pip install huggingface_hub\n!pip install --upgrade accelerate\n!pip install --upgrade bitsandbytes\n!pip install --upgrade torchmetrics\n!pip install --upgrade deepspeed\n'

In [3]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
#from chat_template_utils import get_json_schema
#from llama_cpp import Llama
from utils import *
from peft import *
import datasets
#import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
#from hqq.engine.hf import HQQModelForCausalLM
#from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
#from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)

def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


0it [00:00, ?it/s]

In [4]:
train_dataset = pd.read_excel('train_dataset.xlsx',index_col=0,dtype={'label':str})
#train_dataset = train_dataset.drop_duplicates()
#train_dataset.to_excel('train_dataset.xlsx')
train_dataset


,query,element,label,action,utterance
0,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",email,NaN,NaN
1,Accept additional cookies,"<input id=""accept_all_cookies_button"" messaged...",cookie,NaN,NaN
2,Reject additional cookies,"<input id=""accept_essential_cookies_button"" me...",cookie,NaN,NaN
3,Your Name,"<input aria-describedby=""ph-a11y-email-job-nam...",first name,NaN,NaN
4,Recipient’s Email address,"<input aria-describedby="""" aria-label=""Please ...",email,NaN,NaN
...,...,...,...,...,...
2716,og in,"<button class=""careersite-button careersite-bu...",login,1.0,log in
2717,upload resume,"<a href=""javascript: void(0);"" title=""upload r...",cv,1.0,upload resume
2718,upload fileprivacy policy,"<a href=""https://vertriebskarriere.hansemerkur...",privacy,1.0,privacy policy
2719,back,"<a href=""https://bewerber-innendienst.hansemer...",cancel,1.0,back


In [5]:
l = train_dataset['label'].unique()
id2key = dict(enumerate(l))
key2id = dict(zip(id2key.values(),id2key.keys()))
key2id

{'email': 0,
 'cookie': 1,
 'first name': 2,
 'search': 3,
 'distance': 4,
 'address': 5,
 'country': 6,
 'city': 7,
 'password': 8,
 'last name': 9,
 'phone number': 10,
 'cv': 11,
 'cover letter': 12,
 'job title': 13,
 'zip': 14,
 'username': 15,
 '1': 16,
 'available from': 17,
 'birth': 18,
 'graduation year': 19,
 'company': 20,
 'salary': 21,
 'github': 22,
 'linkedin': 23,
 'English language level': 24,
 'submit': 25,
 'newsletter': 26,
 'civic number': 27,
 'picture': 28,
 'recommender': 29,
 'major': 30,
 'captcha': 31,
 'pronoun': 32,
 'years of experience': 33,
 'house number': 34,
 'find us': 35,
 'transcript': 36,
 'university': 37,
 'skill': 38,
 'login': 39,
 'visa': 40,
 '0': 41,
 '2': 42,
 '3': 43,
 '4': 44,
 '5': 45,
 '6': 46,
 '7': 47,
 '8': 48,
 '9': 49,
 '10': 50,
 '11': 51,
 '12': 52,
 '13': 53,
 'privacy': 54,
 'subscribe': 55,
 'forgot password': 56,
 'twitter': 57,
 'next': 58,
 'cancel': 59,
 'show password': 60,
 'xing': 61,
 'ethnicity': 62,
 'fake': 63,
 '

In [6]:
train_dataset[train_dataset['element'].isna()]

,query,element,label,action,utterance


In [7]:
test_dataset = pd.read_excel('./data/forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)
test_dataset

,query,element,label,sudo_label
0,Personal information\n CV or resume\n Browse C...,"<input accept=""application/pdf,.pdf"" name=""cv""...",cv,login
1,Name *\n,"<input name=""first_name"" placeholder=""First"" r...",first name,first name
2,This field contains invalid characters\n,"<input name=""last_name"" placeholder=""Last"" req...",last name,transcript
3,This field contains invalid characters\n Email...,"<input name=""email"" placeholder=""yourmail@doma...",email,email
4,Phone\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone number,phone number
...,...,...,...,...
95,cover letter\n upload your cover letter\n,"<input accept=""application/msword, application...",cover letter,NaN
96,github\n,"<input name=""custom_attribute_356304"" placehol...",github,NaN
97,ihre e-mail-adresse\n,"<button aria-label=""subscribe for newsletter"" ...",newsletter,NaN
98,(!)\n this field is required\n,"<button id=""uploaddropboxresume"" type=""submit""...",submit,NaN


In [8]:

person = {
    "first name": "majid",
    "last name": "sharghi foroushani",
    "pronoun": "he",
    "birth": "10/08/1997",
    "age": "27",
    "sex": "male",
    "skill": "NLP,LLM",
    "country": "Germany",
    "address": "Philip-9",
    "city": "Erlangen",
    "phone number": "+497773154",
    "available from": "6/24/2024",
    "salary": "10000",
    "German language level": "A1",
    "English language level": "C1",
    "visa": "I do not need visa.",
    "email": "sharghi.majid@gmail.com",
    "password": "123456",
    "find us": "Google",
    "xing": "www.xing.com",
    "job title": "I am a stduent.",
    "university": "FAU",
    "major": "AI",
    "privacy": "privacy",
    "search": "search",
    "submit": "submit",
    "cookie": "cookie",
    "cover letter": "cover letter",
    "linkedin": "www.linkedin.com",
    "github": "www.github.com",
    "login": "login",
    "register": "register",
    "cv": "/pah/to/cv.pdf",
    "twitter": "www.twitter.com",
    "years of experience": "4 years in deep learning",
    "username":"Majidsh97",
    "website language":"english",
    "transcript":"path/to/ts.pdf",
    "company":"company",
    "house number":"9",
    "italian language level":"A2",
    "ethnicity":"white",
    "picture":"path/to/p.jpg",
    "career level":"entry",
    "zip":"91052",
    "newsletter":"newsletter",
    "fake":"fake",
    "question":"question",
    "cancel":"cancel",
    "recommender":"my recommender",
    "website":"my website",
    'workherebefore':"No",
    "next":"next",
    "dropbox":"www.dropbox.com",
    "googledrive":"googledrive",
    "work hours":"8",
    "workduration":"6 months",
    "willing to relocate":"yes",
    "universityenrollment":"path to enrollment letter",
    "subscribe":"subscribe",
    "distance":"50 km",
    "captcha":"captcha",
    "forgot password":"no",
    "graduation year":"2024",
    "civic number":"54864",
    "person title":"Mr",
    "show password":"show password",
    
    
}
# {'years of experience', 'forgot password', 'next', 'username', 'website language', 'latitude', 'distance', 'transcript', 'english language level', 'cancel', 'company', 'longitude', 'house number', 'italian language level', 'title', 'captcha', 'civic number', 'zip', 'recommender', 'newsletter', 'show password', 'ethnicity', 'graduation year', 'job title', 'subscribe', 'picture', 'career level'}




In [9]:
from transformers import AutoModelForSequenceClassification
device = 'cuda'
model_name = 'FacebookAI/xlm-roberta-base'
#model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_name #Qwen/Qwen2-0.5B-Instruct
                                         #,attn_implementation="flash_attention_2"
                                         #,torch_dtype=torch.float16
                                         , num_labels=len(id2key)
                                         ,id2label=id2key
                                         ,label2id=key2id
                                         
                                         , device_map='auto',cache_dir=cache_dir).to('cuda')

model.eval()
#model.config['use_sliding_window']=True
print(model.config)

model

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaConfig {
  "_name_or_path": "FacebookAI/xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "email",
    "1": "cookie",
    "2": "first name",
    "3": "search",
    "4": "distance",
    "5": "address",
    "6": "country",
    "7": "city",
    "8": "password",
    "9": "last name",
    "10": "phone number",
    "11": "cv",
    "12": "cover letter",
    "13": "job title",
    "14": "zip",
    "15": "username",
    "16": "1",
    "17": "available from",
    "18": "birth",
    "19": "graduation year",
    "20": "company",
    "21": "salary",
    "22": "github",
    "23": "linkedin",
    "24": "English language level",
    "25": "submit",
    "26": "newsletter",
    "27": "civic number",
    "28": "picture",
    "29": "recommender",
    "30": "major",

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [10]:
model.generation_config

In [11]:
#model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q_proj','v_proj','k_proj']))
model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['query','key','value']))

#model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q','v','k']))

model.print_trainable_parameters()
model
for n,p in model.named_parameters():
    if p.requires_grad==False:
        print(n)

trainable params: 442,368 || all params: 278,544,460 || trainable%: 0.1588
base_model.model.roberta.embeddings.word_embeddings.weight
base_model.model.roberta.embeddings.position_embeddings.weight
base_model.model.roberta.embeddings.token_type_embeddings.weight
base_model.model.roberta.embeddings.LayerNorm.weight
base_model.model.roberta.embeddings.LayerNorm.bias
base_model.model.roberta.encoder.layer.0.attention.self.query.base_layer.weight
base_model.model.roberta.encoder.layer.0.attention.self.query.base_layer.bias
base_model.model.roberta.encoder.layer.0.attention.self.key.base_layer.weight
base_model.model.roberta.encoder.layer.0.attention.self.key.base_layer.bias
base_model.model.roberta.encoder.layer.0.attention.self.value.base_layer.weight
base_model.model.roberta.encoder.layer.0.attention.self.value.base_layer.bias
base_model.model.roberta.encoder.layer.0.attention.output.dense.weight
base_model.model.roberta.encoder.layer.0.attention.output.dense.bias
base_model.model.roberta

In [14]:
import random


def get_random():
    d = {}
    for k in person.keys():
        d[k] = str(random.randint(1000, 10000))

    return d


# get_random()
def dict_to_str(p):
    s = "{"
    for k, v in p.items():
        s += f'"{k}":"{v}",'
        pass

    s += "}"
    return s


def get_one_chat_template(query,element,utterance="",person_str="", label_str=None,label_action=None,action_history=""):
     
            return f"""CONTENT: {{"{utterance}",{person_str}}}."""+  f"QUERY: {query}"+ f"\nHTML: {element}"
 


def get_one_msg(text, label):
    p = person #get_random()
    strp = ""#dict_to_str(p)
    rc = ""  # random.choice(list(p.keys()))+' '
    d = get_one_chat_template(strp, rc + text,label)#p[label])
    return d


def get_chat():

    d=train_dataset.apply( lambda x :get_one_chat_template(x['query'],x['element'],x['utterance'],"",x['label'],x['action']),axis=1).tolist()

    return d  # ,labels


get_chat()

['CONTENT: {"nan",}.QUERY: Email Address\nHTML: <input id="fieldEmail" name="cm-ukutgu-ukutgu" required="" type="email"/>',
 'CONTENT: {"nan",}.QUERY: Accept additional cookies\nHTML: <input id="accept_all_cookies_button" messagedays="90" name="accept_all_cookies_button" type="submit" value="Accept additional cookies"/>',
 'CONTENT: {"nan",}.QUERY: Reject additional cookies\nHTML: <input id="accept_essential_cookies_button" messagedays="90" name="accept_essential_cookies_button" type="submit" value="Reject additional cookies"/>',
 'CONTENT: {"nan",}.QUERY: Your Name\nHTML: <input aria-describedby="ph-a11y-email-job-name-field-error" aria-label="Please enter name" au-target-id="83" autocomplete="off" data-ph-at-id="name-input" data-ph-id="ph-view1-1569569080710-ph-email-job-v1v0m0ro-iXO62w" key-aria-label="v0m0ro-ph-email-job-v1-view1-yourNameAriaLbl" key-placeholder="v0m0ro-ph-email-job-v1-view1-nameSimpleSecondPlaceholderText" placeholder="Your name" type="text" value.bind="emailJobMo

In [32]:
def get_train_data():
    return tokenizer(get_chat() ,truncation=True,max_length=512)  # ,return_tensors='pt')
    pass

tokenized_data = get_train_data()


no_label = tokenizer([  get_one_chat_template( 
                                                                   str(x['query']),
                                                                   x['element']
                                                                   ) 
                                for i,x in test_dataset.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')

print(tokenized_data[0])
print(tokenizer.decode(tokenized_data[0].ids))
print(len(tokenized_data[0]))
#print(tokenizer.decode(token_ids=no_label[0]))


Encoding(num_tokens=48, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
<s> CONTENT: {"nan",}.QUERY: Email Address HTML: <input id="fieldEmail" name="cm-ukutgu-ukutgu" required="" type="email"/></s>
48


In [31]:
tokenized_data[0].ids

[0,
 6,
 108369,
 20157,
 12,
 10666,
 58,
 4458,
 830,
 8152,
 5,
 53966,
 34388,
 12,
 14886,
 49990,
 19097,
 12,
 4426,
 73,
 7077,
 3447,
 22422,
 28394,
 161994,
 58,
 9351,
 22422,
 3931,
 9,
 10508,
 18,
 1234,
 9,
 10508,
 18,
 1234,
 58,
 56065,
 22422,
 58,
 10644,
 22422,
 25501,
 58,
 64,
 2740,
 2]

In [46]:
dc = DataCollatorWithPadding(tokenizer=tokenizer)
dc(tokenized_data[0])

TypeError: argument of type 'tokenizers.Encoding' is not iterable

In [40]:
from transformers import TrainerCallback
"""
class ChangeDatasetCallback(TrainerCallback):
    def __init__(self):
        #super.__init__()
        pass

    def on_epoch_end(self, args, state, control, **kwargs):
        # Move to the next dataset at the end of each epoch
        #self.current_epoch += 1
        #new_dataset = self.datasets[self.current_epoch % len(self.datasets)]
        kwargs['train_dataloader'].dataset = get_train_data()
        #kwargs['eval_dataloader'].dataset = new_dataset['validation']
"""

class custom_dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.dataset = get_train_data()
        self.len = len(self.dataset)
        pass

    def __getitem__(self, i) :
        if i==self.len-1:
            self.dataset = get_train_data()

        return self.dataset[i]
        pass

    def __len__(self):
        return self.len
        pass

trainer =  Trainer(model,
        train_dataset=tokenized_data,#custom_dataset(),
        data_collator=default_data_collator, #DataCollatorWithPadding(tokenizer=tokenizer),
        tokenizer=tokenizer,
        #compute_metrics=compute_metric,
        #callbacks=[ChangeDatasetCallback()],
        args=TrainingArguments(
          output_dir=output_dir,
          num_train_epochs=2,
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          logging_steps=300,

          #eval_accumulation_steps =2,


        ))

trainer.train()
model = model.eval()

ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,labels,output_attentions,output_hidden_states,return_dict,label,label_ids,labels.

In [ ]:
#trainer.model.save_pretrained(output_dir+'saved_84/')


In [ ]:
questions = [
"use html to create email"
,"""what is your gender?"""
,"""where are you living?(optional)"""
,"what is your salary expectation?*"
,"Which pronouns should we use to address you? (optional)"
,"how is your German?"
,"do you need visa spunsership?"
,"where are you working right now?"
,"what is your birth date?"
,"what are your hobbies?"
,"password"
,"what projects have you done?"
,"are you a text ai?"
,"what experice do you have in network engineering?"
,"what projects have you done in deep learning?"
,"how did you find us?"]

#model.disable_adapter()

x= [get_one_chat_template(""#person_to_str(person)
                          ,q+"""<input type="text" style="" id="custom">""") for q in questions]
tokenized_questions = tokenizer.apply_chat_template(x)

with torch.no_grad():
    
    for i,x in enumerate(tokenized_questions):
            x = torch.tensor([x]).to('cuda')
            o = model.generate(input_ids=x,max_new_tokens=500,do_sample=False)
            
            print(questions[i])
            #decoded = tokenizer.decode(o[0,x.shape[1]+3:])
            decoded = tokenizer.decode(o[0])
            
            print(decoded)
            print('--------------------------------------------------------')
        
    

In [ ]:
model = model.to('cuda')
def get_answer(decoded):
    try:
                #print("ERROR: ",decoded)

                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                return answer
                
    except Exception as e:
                print("ERROR: ",e)
                print(decoded)
                print("---------------------------------")

                
def compute_metric():
    model.eval()
    acc=0
    with torch.no_grad():
        for i in range(len(no_label)):
            x = mydc([no_label[i]])['input_ids'].to('cuda')
            #print(x)
            o = model.generate(input_ids=x,max_new_tokens=20,do_sample=False)
            decoded = tokenizer.decode(o[0,x.shape[1]:])
            #print(decoded)
            #break

            #print(x[0],x[1])
            answer = get_answer(decoded)
            _label = test_dataset.loc[i,'label']
            #_label = person[test_dataset.loc[i,'label']]
            c = answer==_label

            if c:
                acc +=1
            else:
                print("NO_CORRECT:",answer,_label)
                print("---------------------------------")

                pass

    acc/=len(no_label)
    print("acc:",acc)



compute_metric()

In [ ]:
model.eval()
i =11
d = mydc(no_label[i:i+1])['input_ids'].to(device)
d = tokenizer.apply_chat_template( [#{"role":"system","content":""}
                             {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                            ,{"role":"user", "content":'email'}] , return_tensors='pt').to(device)
#d = tokenizer(['json {"answer":""} what is this html element about? <input type="email">'],return_tensors='pt')['input_ids'].to(device)
#print(d)
with torch.no_grad():

    o = model.generate(input_ids = d,max_length = 512,do_sample=True)
    print(tokenizer.decode(o[0]))



In [ ]:
"""x =Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B-Instruct',cache_dir=cache_dir)
model = PeftModelForCausalLM.from_pretrained(x,output_dir+'saved/')"""